## First, run this cell to set up paths and import dependencies

In [ ]:
import os

from tqdm import tqdm

if not os.path.exists(r"./notebooks"):
    %cd ..

from src.audio_processor import AudioProcessor, compute_duration_statistics
from src.audio_dataset_processor import AudioDatasetProcessor
from src.data_processing import SOAAudioClips, save_mean_std, compute_mean_std_from_images
from src.dataset_analysis import duration_statistics
from src.config import VALID_ACCESS_LABELS, TRAIN_DIR, TEST_DIR, VAL_DIR, DATA_DIR, DATASET_DIR

os.makedirs(DATASET_DIR, exist_ok=True)
os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(VAL_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)

## 1. Split all allowed .wav files
We are using [DAPS](https://zenodo.org/records/4660670) dataset. It has several directories available in which there are .wav files of 5 scripts read by 20 speakers. Directories differ from each other with augmentation, which is labeled by `room` and `recording device`. In this cell we are specifying allowed directories, their contents are being discovered and splitted into 3 datasets (training, validation and test). The same speaker with the same script with different augmentation cannot be in the same dataset - `AudioDatasetProcessor` class take care of that. `balance` parameter set to true balances authorized and unauthorized speakers file count in training dataset.

In [ ]:
allowed_directories=['ipadflat_confroom1', 'ipadflat_office1', 'ipad_balcony1', 'ipad_bedroom1', 'ipad_confroom1', 'ipad_confroom2', 'ipad_livingroom1', 'ipad_office1', 'ipad_office2', 'iphone_balcony1', 'iphone_bedroom1', 'iphone_livingroom1']
dataset_processor = AudioDatasetProcessor(DATA_DIR, VALID_ACCESS_LABELS, allowed_directories)
dataset_processor.compute_statistics()
train_set, validate_set, test_set = dataset_processor.get_datasets(balanced=True) # if you want unbalanced set parameter to False

## 2. Duration statistics of .wav files
We first are checking full clips duration statistics.

In [ ]:
soa_train_full_clips = SOAAudioClips(train_set)
soa_test_full_clips = SOAAudioClips(validate_set)
soa_val_full_clips = SOAAudioClips(test_set)

print("\nDataset Statistics:")
print("Training set:")
print(duration_statistics(soa_train_full_clips.clips))

print("Validation set:")
print(duration_statistics(soa_test_full_clips.clips))

print("Test set:")
print(duration_statistics(soa_val_full_clips.clips))

## 3. Split into few seconds clips
Now we decide to split full clips into few seconds subclips, they are filtered so only clips with more than `0.5` of recording has speech detected by `webrtcvad`. We are doing this to establish the same size of input for CNN.

In [ ]:
CLIP_DURATION_SECONDS = 3

def process_audio_clips(audio_processor, soa_full_clips, output_dir, clip_duration=CLIP_DURATION_SECONDS):
    """Processes and splits audio clips, saves spectrograms, and computes statistics."""
    all_clips = []
    for file_path, full_clip in tqdm(soa_full_clips, desc="Processing audio clips"):
        clips = audio_processor.split_audio(full_clip, clip_duration)
        speech_clips = audio_processor.filter_speech_clips(clips)
        all_clips.extend(speech_clips)

        for i, clip in enumerate(speech_clips):
            spectrogram = audio_processor.create_spectrogram(clip)
            output_path = os.path.join(
                output_dir, f"{os.path.basename(file_path).split('.')[0]}_{i}_clip.png"
            )
            audio_processor.save_spectrogram(spectrogram, output_path)

    print(compute_duration_statistics(all_clips))


Then after processing we save log mel grayscale spectrograms into separate directories for every dataset.

In [ ]:
audio_processor = AudioProcessor()

print("Preprocessed Train Dataset:")
process_audio_clips(audio_processor, soa_train_full_clips, TRAIN_DIR)

print("\nPreprocessed Validation Dataset:")
process_audio_clips(audio_processor, soa_val_full_clips, VAL_DIR)

print("\nPreprocessed Test Dataset:")
process_audio_clips(audio_processor, soa_test_full_clips, TEST_DIR)

## 4. Mean and standard deviation of training dataset
It is important to normalize data for our Neural Networks. It creates better distribution. It should smoothen loss function plane (so it would be easier to find global minimum). We use only training dataset for this purpose and save it to JSON file next to datasets' image directories.

Effect of normalization for example CNN can be seen [here](https://wandb.ai/mytkom-warsaw-university-of-technology/iml/reports/Effect-of-normalization-input-for-TutorialCNN--VmlldzoxMDUxMTI1OQ?accessToken=s67utpfjryb4um1240bd56i51zo5oy2bj0gbaqqz79z3hnnabkub1rdhsamhwd2v).

In [ ]:
mean, std = compute_mean_std_from_images(TRAIN_DIR)
print(f"Mean: {mean}, Standard deviation: {std}")
save_mean_std(mean, std, f"{DATASET_DIR}/scaling_params.json")